In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv("C:/Users/Admin/Desktop/Data/Assignments New/Random Forest/Fraud_check (2).csv")
data.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [4]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
from urllib.request import urlopen 


In [10]:
data.shape

(600, 6)

In [12]:
data.isnull().sum()

undergrad     0
marital       0
population    0
experience    0
urban         0
income        0
dtype: int64

In [11]:
data.describe()

,population,experience
count,600.000000,600.000000
mean,108747.368333,15.558333
std,49850.075134,8.842147
min,25779.000000,0.000000
25%,66966.750000,8.000000
50%,106493.500000,15.000000
75%,150114.250000,24.000000
max,199778.000000,30.000000


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   undergrad   600 non-null    object
 1   marital     600 non-null    object
 2   population  600 non-null    int64 
 3   experience  600 non-null    int64 
 4   urban       600 non-null    object
 5   income      600 non-null    object
dtypes: int64(2), object(4)
memory usage: 28.2+ KB


In [7]:
data["income"]="<=30000"
data.loc[data["Taxable.Income"]>=30000,"income"]="Good"
data.loc[data["Taxable.Income"]<=30000,"income"]="Risky"

In [8]:
##Droping the Taxable income variable
data.drop(["Taxable.Income"],axis=1,inplace=True)

In [22]:
data.rename(columns={"Undergrad":"undergrad","Marital.Status":"marital","City.Population":"population","Work.Experience":"experience","Urban":"urban"},inplace=True)
## As we are getting error as "ValueError: could not convert string to float: 'YES'".
## Model.fit doesnt not consider String. So, we encode

In [23]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
for column_name in data.columns:
    if data[column_name].dtype == object:
        data[column_name] = le.fit_transform(data[column_name])
    else:
        pass

In [25]:
##Splitting the data into featuers and labels
features = data.iloc[:,0:5]
labels = data.iloc[:,5]

In [27]:
## Collecting the column names
colnames = list(data.columns)
predictors = colnames[0:5]
target = colnames[5]

In [28]:
##Splitting the data into train and test

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(features,labels,test_size = 0.2,stratify = labels)

In [29]:
##Model building
from sklearn.ensemble import RandomForestClassifier as RF
model = RF(n_jobs = 3,n_estimators = 15, oob_score = True, criterion = "entropy")
model.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=15, n_jobs=3,
                       oob_score=True)

In [30]:
model.estimators_
model.classes_
model.n_features_
model.n_classes_

2

In [31]:
model.n_outputs_

1

In [33]:
model.oob_score_

0.6979166666666666

In [34]:
##Predictions on train data
prediction = model.predict(x_train)

In [36]:
##Accuracy
# For accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train,prediction)
accuracy

0.99375

In [37]:
np.mean(prediction == y_train)

0.99375

In [38]:
##Confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_train,prediction)

In [39]:
##Prediction on test data
pred_test = model.predict(x_test)

In [40]:
##Accuracy
acc_test =accuracy_score(y_test,pred_test)

In [42]:
!pip install pydotplus

  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24566 sha256=c998a2d9a063fecd76492f15905d240109f4557b96ac1878d0d6a86eef3cde95
  Stored in directory: c:\users\mohan kumar\appdata\local\pip\cache\wheels\fe\cd\78\a7e873cc049759194f8271f780640cf96b35e5a48bef0e2f36
Successfully built pydotplus


In [43]:
from sklearn.datasets import load_iris
from sklearn.tree import export_graphviz
import pydotplus
#from sklearn.externals.six import StringIO

In [44]:
tree = model.estimators_[5]

In [51]:
from io import StringIO
dot_data = StringIO()
dot_data

In [47]:
export_graphviz(tree,out_file = dot_data, filled = True,rounded = True, feature_names = predictors ,class_names = target,impurity =False)

In [50]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph

In [49]:
## Creating pdf and png file the selected decision tree
graph.write_pdf('fraudrf.pdf')
graph.write_png('fraudrf.png')

InvocationException: GraphViz's executables not found